In [24]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langsmith import traceable
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda


In [25]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = TextLoader("../files/chapter_three.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever()


In [26]:
# 프롬프트 템플릿 정의
template = """다음 정보를 사용하여 질문에 답변하세요:

{context}

질문: {question}
이전 대화:
{chat_history}

답변:"""

PROMPT = PromptTemplate(
    input_variables=["context", "question", "chat_history"],
    template=template
)

In [27]:
# 메모리 초기화
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [28]:
# 체인 수동 구현
def retrieve_docs(query):
    return retriever.get_relevant_documents(query)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def generate_answer(input_dict):
    context = input_dict["context"]
    question = input_dict["question"]
    chat_history = input_dict["chat_history"]
    
    prompt = PROMPT.format(context=context, question=question, chat_history=chat_history)
    return llm.invoke(prompt)

In [30]:
chain = (
    {
        "context": retrieve_docs | format_docs,
        "question": RunnablePassthrough(),
        "chat_history": lambda x: memory.load_memory_variables({})["chat_history"]
    }
    | RunnableLambda(generate_answer)
)

TypeError: unsupported operand type(s) for |: 'function' and 'function'